In [26]:
## coding: utf-8
# データベースを作る
# 計測用信号を読み込む
# 測位する

#360°のデータベース作成
181*81*144*7
#2倍にアップサンプリング



import wave
import scipy as sp
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from scipy.fftpack import fft
import scipy.signal as sg
from scipy import signal
import pandas as pd
import scipy.interpolate as itpl
import resampy

def chirp_exp(numSamples, chirpLen_s, start_Hz, stop_Hz, phase_rad):
    # チャープの生成関数,chirpaは生成したチャープ信号の配列を返す
    times_s = np.linspace(0, chirpLen_s, numSamples) # Chirp times.
    k = (stop_Hz - start_Hz) / chirpLen_s # Chirp rate.
    sweepFreqs_Hz = (start_Hz + k/2. * times_s) * times_s
    chirpa = np.array(np.exp(phase_rad*1j-2*np.pi*1j*sweepFreqs_Hz))
    return chirpa

def create_db():
    sound_db = np.zeros(144*7*181*81*2).reshape(181,81,144*7*2)
    sound_db[:,:,:] = np.nan
    speaker_deg = 0
    for j in range(0,7):
        mic_deg = -40
        for i in range(0,9):
            #整形済み信号読み込み
            signal = np.loadtxt('s'+str(speaker_deg)+'m'+str(mic_deg)+'.txt')
            signal_fft = np.abs(fft(signal))
            chirps_up =  resampy.resample(signal_fft, 144, 144*2, filter='kaiser_fast')
            sound_db[speaker_deg,mic_deg+40,:] = chirps_up
            mic_deg += 10
        speaker_deg +=30
    #ここから秋間補間，マイク方位で補間した後スピーカ角度で補間
    Akima = np.zeros(144*7*181*81*2).reshape(181,81,144*7*2)
    Akima[:,:,:] = np.nan
    #マイク方位で補間
    count = 0
    for i in range(0,7):
        sound_db_d = pd.DataFrame(sound_db[count,:,:])
        sound_db_d.astype('float64')
        Akima[count,:,:] = sound_db_d.interpolate('akima')
        count += 30
    #スピーカ角度で補間
    for j in range(0,81):
        sound_db_d = pd.DataFrame(Akima[:,j,:])
        sound_db_d.astype('float64')
        Akima[:,j,:] = sound_db_d.interpolate('akima')
    #Akima = pd_Zrf_d.interpolate('akima')
    print('Akima done!!')
    return Akima
    
# 残差平方和
def rss(y, t):
    f = np.sum((y-t).T*(y-t))
    return f

def normalzie(x, amin=0, amax=1):
    xmax = x.max()
    xmin = x.min()
    if xmin == xmax:
        return np.ones_like(x)
    return (amax - amin) * (x - xmin) / (xmax - xmin) + amin
#normalized_x = normalzie(x, amin=-1, amax=1)
    
def estimate(db,file):
    #データベースを読み込んで、残差平方和を計測
    rss_db = np.zeros(181*81).reshape(181,81)
    #print(rss_db)
    for j in range(0,181):
        for i in range(0,81):
            rss_ans = rss(normalzie(db[j,i],0,1),normalzie(file[:],0,1))
            rss_db[j,i] = rss_ans
    estimate_mic_speaker = np.unravel_index(np.argmin(rss_db), rss_db.shape)
    estimate_mic_speaker = list(estimate_mic_speaker)
    chirp = chirp_exp(144*2,0.003,1000,1100,np.pi*0.5)
    #print(db[estimate_mic_speaker,0:144])
    #print(file[0:144])
    #print(135.0*max(np.abs(sg.correlate(db[estimate_mic_speaker[0],estimate_mic_speaker[1],0:144],chirp, mode='same', method='auto'))))
    #print(max(np.abs(sg.correlate(file[0:144],chirp, mode='same', method='auto'))))
    print('距離'+str((135.0*max(np.abs(sg.correlate(db[estimate_mic_speaker[0],estimate_mic_speaker[1],0:144],chirp, mode='same', method='auto'))))/max(np.abs(sg.correlate(file[0:144],chirp, mode='same', method='auto')))))
    #mf_out = sg.correlate(estimate_mic_speaker,chirp, mode='same', method='auto')
    estimate_mic_speaker[1] -= 40
    print('(推定スピーカー角度,推定マイク角度)'+str(estimate_mic_speaker))
    
if __name__ == '__main__':
    db1 = create_db()
    #整形済み測位信号読み込み
    signal = np.loadtxt('s0m-40new.txt')
    #chirps_up1 = np.abs(fft(signal))
    chirps_up1 =  resampy.resample(np.abs(fft(signal)), 144, 144*2, filter='kaiser_fast')
    # 最小二乗法によって測位
    estimate(db1,chirps_up1)

Akima done!!
距離197.1768709672858
(推定スピーカー角度,推定マイク角度)[179, -40]


In [ ]:
(スピーカー角度,マイク角度,距離cm)[0,-25,130]
(推定スピーカー角度,推定マイク角度,距離)[1, -28,139]

(スピーカー角度,マイク角度,距離cm)45,-25,130
(推定スピーカー角度,推定マイク角度,距離)[43, -29,185]

(スピーカー角度,マイク角度,距離cm)45,25,130
(推定スピーカー角度,推定マイク角度,距離)[45, 19,162]

(スピーカー角度,マイク角度,距離cm)0,40,200
(推定スピーカー角度,推定マイク角度,距離)[170, 0,574]

(スピーカー角度,マイク角度,距離cm)0,-40,200
(推定スピーカー角度,推定マイク角度,距離)[179, -40,197]

(スピーカー角度,マイク角度,距離cm)25,-30,200
(推定スピーカー角度,推定マイク角度,距離)[37, -38,268]

(スピーカー角度,マイク角度,距離cm)25,30,200
(推定スピーカー角度,推定マイク角度,距離)[175, 23,357]

(スピーカー角度,マイク角度,距離cm)170,25,150
(推定スピーカー角度,推定マイク角度,距離)[3, -39,169]

(スピーカー角度,マイク角度,距離cm)170,-25,150
(推定スピーカー角度,推定マイク角度,距離)[6, 16,109]